# DM-25824: Rotator/CCW synchronization test

This notebook contains code to drive the CCW-Rotator integration test. There is an accompanying notebook that shows how to query the EFD for the data generated here (CCW-Rotator-EFD.ipynb) and also a script that can be launched on the ScriptQueue to perform a similar task. 

https://jira.lsstcorp.org/browse/DM-25824

In [ ]:
import sys
import asyncio
import logging

import numpy as np

import astropy.units as u
from astropy.coordinates import Angle

from lsst.ts import salobj

from lsst.ts.observatory.control.maintel import MTCS

In [ ]:
stream_handler = logging.StreamHandler(sys.stdout)

logger = logging.getLogger()
logger.addHandler(stream_handler)
logger.level = logging.DEBUG

In [ ]:
mtcs = MTCS()

In [ ]:
await mtcs.start_task

In [ ]:
mtcs.components

## Ignoring components because they are not important for this test.

This allow the test to run even if the components are not available.

In [ ]:
mtcs.check.mtaos = False
mtcs.check.mtm1m3 = False
mtcs.check.mtm2 = False
mtcs.check.hexapod_1 = False
mtcs.check.hexapod_2 = False
mtcs.check.dome = False
mtcs.check.mtdometrajectory = False

## Check heartbeat from NewMTMount component.

In [ ]:
await mtcs.rem.newmtmount.evt_heartbeat.next(flush=True)

## Ensuring components are in ENABLED state

In [ ]:
await asyncio.sleep(5.)
for comp in mtcs.components:
    if not getattr(mtcs.check, comp):
        continue
    
    try:
        state = await mtcs.get_state(comp)
        print(f"{comp}: {state!r}")
    except asyncio.TimeoutError:
        print(f"Failed to get status for {comp}.")
        pass

Sometimes it happens that the Rotator is in `FAULT` state. Before enabling Rotator we need to send the `clearError` command.

In [ ]:
if (await mtcs.get_state("rotator")) == salobj.State.FAULT:
    print("Rotator: clear error.")
    await mtcs.rem.rotator.cmd_clearError.start()

Now put all component in `ENABLED` state. This will not affect the components that are already enabled.

In [ ]:
await mtcs.enable(
    {
        'mtmount': '',
        'mtptg': '',
        'rotator': ''
    }
                 )

Need to enable Camera Cable Wrap tracking. 

In [ ]:
await asyncio.sleep(30.)
await mtcs.rem.newmtmount.cmd_enableCameraCableWrapTracking.start()

## Execute first test

This test will randomly select a set of angles between 0 and 80 and move the Rotator from + to -, to make sure that the CCW can cope with both long and short slews.

In order to allow this notebook to run independently of the time, we get the current time information provided by the pointing and compute appropriate coordinates.

The `timeAndDate` topic provided by the pointing contains the Local Sidereal Time, that can be used as a good indication of `RA`.

In [ ]:
time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.aget()
ra = Angle(time_and_date.lst, unit=u.hourangle)
await mtcs.slew_icrs(ra=ra, dec=dec, rot_sky=-180., stop_before_slew=False)

await asyncio.sleep(30.)

for deltaA in np.random.random(10)*80.:
    for i in range(5):

        time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.aget()
        ra = Angle(time_and_date.lst, unit=u.hourangle)
        mtcs.rem.rotator.evt_inPosition.flush()
        await mtcs.slew_icrs(ra=ra, dec=dec, rot_sky=-180.-deltaA, stop_before_slew=False)

        await asyncio.sleep(15.)

        time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.aget()
        ra = Angle(time_and_date.lst, unit=u.hourangle)
        mtcs.rem.rotator.evt_inPosition.flush()
        await mtcs.slew_icrs(ra=ra, dec=dec, rot_sky=-180.+deltaA, stop_before_slew=False)
        
        await asyncio.sleep(15.)


time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.aget()
ra = Angle(time_and_date.lst, unit=u.hourangle)
await mtcs.slew_icrs(ra=ra, dec=dec, rot_sky=-180., stop_before_slew=False)


## Execut second test

This test consists of starting a long slew, waiting for the Rotator to move for some time and then initiate a new slew in the other direction, without stoping.

In [ ]:
time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.aget()
ra = Angle(time_and_date.lst, unit=u.hourangle)
await mtcs.slew_icrs(ra=ra, dec=dec, rot_sky=-180., stop_before_slew=True)

await asyncio.sleep(10.)

time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.aget()
ra = Angle(time_and_date.lst, unit=u.hourangle)
try:
    await mtcs.slew_icrs(ra=ra, dec=dec, rot_sky=-180.+80, stop_before_slew=False, slew_timeout=5.)
except asyncio.TimeoutError:
    pass

time_and_date = await mtcs.rem.mtptg.tel_timeAndDate.aget()
ra = Angle(time_and_date.lst, unit=u.hourangle)
await mtcs.slew_icrs(ra=ra, dec=dec, rot_sky=-180.+10., stop_before_slew=False)
